### Assignment 1: Automatic pieces detection and quantification in thermographic images


#### Steps:
1. Load images  
2. Preprocess (grayscale, normalization, smoothing)  
3. Threshold - binary mask  
4. Morphological cleaning  
5. Contour detection  
6. Filter using area, circularity, and “touching border” test  
7. Draw results, count pieces, and save annotated images + CSV file  

### Import and Setup

In [2]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (8,6)

IMAGE_DIR = "Images"    
OUTPUT_DIR = "output"  

os.makedirs(OUTPUT_DIR, exist_ok=True)

print("Using image directory:", IMAGE_DIR)
print("Output directory ready:", OUTPUT_DIR)


Using image directory: Images
Output directory ready: output


#### Functions

In [4]:
# 1. Preprocessing

def preprocess(img):
    """Normalize contrast and reduce noise."""
    img = img.astype(np.float32)
    img -= img.min()
    if img.max() > 0:
        img = (img / img.max()) * 255
    img = img.astype(np.uint8)

    blurred = cv2.GaussianBlur(img, (5,5), 0)
    return blurred


def binarize(img):
    """Try Otsu threshold, fallback to adaptive."""
    t, th = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    if t < 5:   # bad threshold
        th = cv2.adaptiveThreshold(
            img, 255,
            cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
            cv2.THRESH_BINARY,
            51, -10
        )
    return th


def clean_mask(mask):
    """Use morphological filters to clean noise and fill holes."""
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    opened = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    closed = cv2.morphologyEx(opened, cv2.MORPH_CLOSE, kernel, iterations=2)

    # Fill holes
    contours, _ = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    filled = np.zeros_like(mask)
    cv2.drawContours(filled, contours, -1, 255, thickness=cv2.FILLED)
    return filled


def touches_border(bbox, shape, margin=2):
    x, y, w, h = bbox
    H, W = shape[:2]
    
    if x <= margin or y <= margin or (x + w) >= (W - margin) or (y + h) >= (H - margin):
        return True
    return False


def circularity(cnt):
    area = cv2.contourArea(cnt)
    perim = cv2.arcLength(cnt, True)
    if perim == 0:
        return 0
    return 4 * np.pi * (area / (perim * perim))
